In [9]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
)

In [10]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is the meaning of life? Give a short answer.",
        }
    ],
    model="gpt-4o",
)
print(chat_completion.choices[0].message.content)

The meaning of life is subjective and can vary greatly from person to person. It often involves finding purpose, seeking happiness, forming connections with others, and contributing to something greater than oneself.
